In [1]:
from urllib.request import urlopen as uReq
import urllib.request
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta, date
import csv
import re
import os
import time
from urllib.request import Request, urlopen

my_url = 'http://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings'


In [5]:
#creating backwards time list
datelist = pd.date_range(end=pd.datetime.today(), periods=3, freq='M', normalize=bool)
datelist = datelist.strftime('%Y-%m').tolist()
string1= "&month="
myurls=[]
for x in datelist:
    myurls.append(my_url+string1+str(x))
myurls.append(my_url)
#getting URLS


In [6]:
def make_soup(url):
    thepage=urllib.request.urlopen(url)
    soupdata=soup(thepage,"html.parser")
    return soupdata

In [8]:
#getting basic data
ipodatasaved=[]
newurls=[]  
for eachurl in myurls:
    soup1= make_soup(eachurl)  
    time.sleep(3)
    for record in soup1.findAll("div",{"class":"genTable"})[:1]:
        if record==['\r', '\t\t\t\tThere is no data for this month, please select a prior month.\r', '\t\t\t']:
            continue
        else:
            for data in record.findAll('tr')[1:]:
                ipodata=[]

                
                for element in data.findAll('td'):
                    if element.text == None:
                        continue
                    else:
                        (ipodata.append(element.text))
                    
                    if element.a == None:
                        continue
                    else:
                        newurls.append(element.a['href'])
    
                ipodatasaved.append(ipodata)
       

In [9]:
ipodatasaved3= np.reshape(ipodatasaved,(-1,7))
df6= pd.DataFrame(ipodatasaved3, columns=("company_name", "symbol"," market", "price", 'shares', 'offer_amount', 'date_priced'))



approved=['markets/ipos/company/']
newurls1 = [url for url in newurls if any(sub in url for sub in approved)]
newurls2=pd.DataFrame(newurls1)
#getting urls for each stock

financeurls=[]
fin="?tab=financials"
for x in newurls1:
    financeurls.append(x+fin)
# financeurls
financeurls=pd.DataFrame(financeurls)


In [10]:
#finding lockup expiration
lockup=[]
shares=[]

for eachurl in newurls1: 
    soup1= make_soup(eachurl) 
    time.sleep(3)
    for record in soup1.findAll("div",{"class":"genTable"})[:1]:
            for x in record.findAll('td'):
                if x.find(text="Shares Outstanding")!= None:
                    #print(x.findNextSibling())
                    shares.append(x.findNextSibling().text)
                else:
                    pass
                
                if x.find(text="Lockup Expiration")!=None:
                    lockup.append(x.findNextSibling().text)
                else:
                    continue  



<td>2,446,582</td>
<td>31,862,017</td>
<td>13,640,982</td>
<td>2,600,000</td>
<td>25,000,000</td>
<td>25,250,000</td>
<td>35,537,500</td>
<td>4,500,000</td>
<td>16,732,925</td>
<td>16,918,354</td>
<td>9,420,765</td>
<td>68,021,702</td>
<td>12,000,000</td>
<td>26,114,993</td>
<td>7,678,621</td>
<td>8,750,000</td>
<td>3,100,000</td>
<td>24,000,000</td>
<td>25,600,000</td>
<td>84,457,323</td>
<td>16,250,000</td>
<td>30,230,319</td>
<td>11,939,176</td>
<td>328,948,088</td>
<td>4,750,000</td>
<td>20,000,000</td>
<td>25,000,000</td>
<td>12,875,000</td>
<td>11,980,000</td>
<td>19,800,000</td>
<td>23,750,000</td>
<td>15,597,624</td>
<td>6,500,000</td>
<td>9,424,574</td>
<td>17,060,631</td>
<td>23,189,901</td>
<td>7,243,024</td>
<td>7,419,874</td>
<td>63,276,910</td>
<td>23,909,025</td>
<td>20,000,000</td>
<td>19,400,000</td>
<td>68,365,044</td>
<td>6,497,500</td>
<td>11,219,441</td>
<td>46,875,000</td>


In [11]:

lockup1=pd.DataFrame(lockup)
shares1=pd.DataFrame(shares)


In [16]:
#getting revenue and profit
# url=https://www.nasdaq.com/markets/ipos/company/iqiyi-inc-1046890-86160?tab=experts
revenue=[]
profit=[]

for urls in financeurls[0]:
    soup2=make_soup(urls)
    time.sleep(3)
    for record in soup2.findAll("div",{"class":"genTable"}):
        for x in record.findAll('td'):
            if x.find(text="Revenue")!= None:
                revenue.append(x.findNextSibling().text)
            else:
                pass
                
            if x.find(text="Net Income")!=None:
                profit.append(x.findNextSibling().text)
            else:
                continue


In [47]:

revenue1=pd.DataFrame(revenue)
profit1=pd.DataFrame(profit)


In [68]:
ipodatasaved4= np.reshape(ipodatasaved3,(-1,7))
df6= pd.DataFrame(ipodatasaved4, columns=("company_name", "symbol"," market", "price", 'shares', 'offer_amount', 'date_priced'))

df7 =lockup1.join(shares1,lsuffix='_Lockupdate')
df7=df7.join(revenue1,lsuffix='_Sharesout')
df7=df7.join(profit1,lsuffix ='_Revenue')
df7.columns=['Lockupdate','Sharesoutstand','Revenue','Profit']
df8=df6.join(df7)

